In [1]:
import pandas as pd
import os
import regex as re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [2]:
path = "/Users/aditya/Documents/GitHub/tegger-challenge/bbc/"
contents = []
themes = []
for root, dirs, files in os.walk(path):
    for file in files:
        if(re.match(r"\d*\.txt", file)):
            f = open(os.path.join(root, file), 'r', encoding='latin1')
            # string = f.read()
            contents.append(f.read())
            themes.append(root[len(path):])

In [3]:
data = pd.DataFrame()
data['all'] = contents
data['theme'] = themes
data['title'] = data['all'].apply(lambda x: x[:x.find('\n')])
data['contents'] = data['all'].apply(lambda x: x[x.find('\n')+2:])

In [4]:
data.head()

,all,theme,title,contents
0,Musicians to tackle US red tape\n\nMusicians' ...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...
1,"U2's desire to be number one\n\nU2, who have w...",entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar..."
2,Rocker Doherty in on-stage fight\n\nRock singe...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...
3,Snicket tops US box office chart\n\nThe film a...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...
4,Ocean's Twelve raids box office\n\nOcean's Twe...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin..."


In [5]:
g20 = "Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Republic of Korea, Korea, Mexico, Russia, Saudi Arabia, Arabia, South Africa, Africa, Turkey, United Kingdom, UK, United States, America, USA, European Union, EU, Europe".lower()
g20 = g20.split(", ")
g20

['argentina',
 'australia',
 'brazil',
 'canada',
 'china',
 'france',
 'germany',
 'india',
 'indonesia',
 'italy',
 'japan',
 'republic of korea',
 'korea',
 'mexico',
 'russia',
 'saudi arabia',
 'arabia',
 'south africa',
 'africa',
 'turkey',
 'united kingdom',
 'uk',
 'united states',
 'america',
 'usa',
 'european union',
 'eu',
 'europe']

In [6]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def preprocess_non(non_word):
    non_word = non_word.lower()
    non_word = "".join([char for char in non_word if char not in string.punctuation])
    if (non_word in stop_words or non_word == ""):
        return("")
    non_word = lemmatizer.lemmatize(non_word)
    return(non_word)

def preprocess_g20(g_word):
    g_word = g_word.upper()
    g_word = "*"+g_word
    return(g_word)

def preprocess(entry):
    entry = entry.replace("US", "USA")
    entry = word_tokenize(entry)
    entry = [preprocess_g20(word) if word.lower() in g20 else preprocess_non(word) for word in entry]
    entry = " ".join(entry)
    entry = re.sub(' +', ' ', entry)
    return(entry)

In [7]:
data['all'] = data['all'].apply(preprocess)

In [8]:
data

,all,theme,title,contents
0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...
1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar..."
2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...
3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...
4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin..."
...,...,...,...,...
2220,warning window word file writing microsoft wor...,tech,Warning over Windows Word files,Writing a Microsoft Word document can be a dan...
2221,fast lift rise record book two highspeed lift ...,tech,Fast lifts rise into record books,Two high-speed lifts at the world's tallest bu...
2222,nintendo add medium playing d nintendo releasi...,tech,Nintendo adds media playing to DS,Nintendo is releasing an adapter for its DS ha...
2223,fast moving phone virus appear security firm w...,tech,Fast moving phone viruses appear,Security firms are warning about several mobil...


In [10]:
for each in g20:
    data[each] = [1 if "*"+each.upper() in row['all'].split(" ") else 0 for index, row in data.iterrows()] 

data['republic of korea'] = (data['republic of korea']+data['korea']).apply(lambda x: 1 if x>=1 else 0)
data['saudi arabia'] = (data['saudi arabia']+data['arabia']).apply(lambda x: 1 if x>=1 else 0)
data['south africa'] = (data['south africa']+data['africa']).apply(lambda x: 1 if x>=1 else 0)
data['united kingdom'] = (data['united kingdom']+data['uk']).apply(lambda x: 1 if x>=1 else 0)
data['united states'] = (data['united states']+data['usa'] + data['america']).apply(lambda x: 1 if x>=1 else 0)
data['european union'] = (data['european union']+data['eu'] + data['europe']).apply(lambda x: 1 if x>=1 else 0)

data.drop(columns=['korea', 'arabia', 'africa', 'uk', 'america' , 'usa', 'eu', 'europe'], inplace=True)

In [11]:
data.to_csv('/Users/aditya/Documents/GitHub/tegger-challenge/data.csv')